In [1]:
from rdflib import Graph as RDFGraph
# import matplotlib.pyplot as plt
import statistics
import collections
import pandas as pd
from six.moves.urllib.parse import urldefrag
from pathlib import Path
from os import listdir 
from os.path import isfile, join, abspath, basename
import json
import xbrl2rdf
import re

In [2]:
# Preliminaries

# data/taxonomies should contain: EBA_CRD_IV_XBRL_2.8_Dictionary_2.8.1.1.zip
#                                 EBA_CRD_IV_XBRL_2.8_Reporting_Frameworks_2.8.1.1.zip
# data/instances should contain: DUMMYLEI123456789012_GB_FINREP020202_FINREP9CONIFRS_2018-12-31_20180802120230000.xbrl

# These files can be downloaded from 
# https://www.eba.europa.eu/risk-analysis-and-data/reporting-frameworks/reporting-framework-2.8

# if you want to start over then delete 
# data/rdf/taxonomy/EBA directories
# data/rdf/data/instance.ttl
# data/rdf/preloads.json

# create ttl file of instance

In [3]:
DATA_URL = join("..", "data", "instances", "DUMMYLEI123456789012_GB_FINREP020202_FINREP9CONIFRS_2018-12-31_20180802120230000.xbrl")

OUTPUT_DIR = join("..", "data", "rdf")

# set up output directories
Path(join(OUTPUT_DIR, "data")).mkdir(parents=True, exist_ok=True)

# Process instance and taxonomy (if done the first time then it may take a while)
xbrl2rdf.MainProcessor(DATA_URL, OUTPUT_DIR, 1)

# After this step you have
# the XBRL dictionary in turtle format: data\rdf\taxonomies\EBA_CRD_IV_XBRL_2.8_Dictionary_2.8.1.1
# the XBRL reporting framework in turtle format: data\rdf\taxonomies\EBA_CRD_IV_XBRL_2.8_Reporting_Frameworks_2.8.1.1
# the sample XBRL instance: data\rdf\data\DUMMYLEI123456789012_GB_FINREP020202_FINREP9CONIFRS_2018-12-31_20180802120230000.ttl

# Note:
# only the taxonomy items that are used in the instance are converted

0

# Read ttl

In [4]:
# Here we load in the turtle-files in an in-memory RDFLib Graph (a graph database would be better)

# Initialize graph
g = RDFGraph()

# RDF graph loading

# instance data
data = "../data/rdf/data/DUMMYLEI123456789012_GB_FINREP020202_FINREP9CONIFRS_2018-12-31_20180802120230000.ttl"
g.parse(data, format='turtle')

# taxonomy data reporting framework
taxo = "../data/rdf/taxonomies/EBA_CRD_IV_XBRL_2.8_Reporting_Frameworks_2.8.1.1/"
taxo_ttls = [join(taxo, f) for f in listdir(taxo) if isfile(join(taxo, f))]
for ttl in taxo_ttls:
    try:
        g.parse(ttl, format='turtle')
    except:
        print("Parse error: " + ttl)

# taxonomy data dictionary
taxo = "../data/rdf/taxonomies/EBA_CRD_IV_XBRL_2.8_Dictionary_2.8.1.1/"
taxo_ttls = [join(taxo, f) for f in listdir(taxo) if isfile(join(taxo, f))]
for ttl in taxo_ttls:
    try:
        g.parse(ttl, format='turtle')
    except:
        print("Parse error: " + ttl)
        
print("RDFLib Graph loaded successfully with {} triples".format(len(g)))
# there might be some parse errors -> something is wrong in the ttl file (for example not a proper triple)
# this should be fixed

Parse error: ../data/rdf/taxonomies/EBA_CRD_IV_XBRL_2.8_Dictionary_2.8.1.1/error-formatting-f7b727a941ab2a3cb79254bd682d403c.ttl
RDFLib Graph loaded successfully with 1187724 triples


# Tables

In [5]:
q = """
  SELECT ?a
  WHERE {
    ?a xl:type table:table .
  }"""
tables = [str(row[0]) for row in g.query(q)]
tables.sort()
tables[0:5]

['http://www.eba.europa.eu/eu/fr/xbrl/crr/fws/finrep/cir-680-2014/2018-03-31/tab/f_03.00/f_03.00-rend.xml#eba_tF_03.00',
 'http://www.eba.europa.eu/eu/fr/xbrl/crr/fws/finrep/cir-680-2014/2018-03-31/tab/f_05.01/f_05.01-rend.xml#eba_tF_05.01',
 'http://www.eba.europa.eu/eu/fr/xbrl/crr/fws/finrep/cir-680-2014/2018-03-31/tab/f_07.01/f_07.01-rend.xml#eba_tF_07.01',
 'http://www.eba.europa.eu/eu/fr/xbrl/crr/fws/finrep/cir-680-2014/2018-03-31/tab/f_08.02/f_08.02-rend.xml#eba_tF_08.02',
 'http://www.eba.europa.eu/eu/fr/xbrl/crr/fws/finrep/cir-680-2014/2018-03-31/tab/f_12.01.a/f_12.01.a-rend.xml#eba_tF_12.01.a']

## Concept labels

In [6]:
q = """
  SELECT ?t ?label
  WHERE {
    ?x1 <http://www.xbrl.org/2003/arcrole#concept-label> [xl:from ?t ;
                                xl:to [rdf:value ?label]] .
  }"""
concept_labels = {str(row[0]): str(row[1]) for row in g.query(q)}

In [7]:
pd.DataFrame(index = concept_labels.keys(), data = concept_labels.values()).to_csv("finrep_280-concept-labels.csv")

In [8]:
q = """
  SELECT ?t ?label
  WHERE {
    ?x1 <http://xbrl.org/arcrole/2008#element-label> [xl:from ?t ;
                                xl:to [rdf:value ?label]] .
  }"""
element_labels = {str(row[0]): str(row[1]) for row in g.query(q)}

In [9]:
pd.DataFrame(index = element_labels.keys(), data = element_labels.values()).to_csv("finrep_280-element-labels.csv")

In [10]:
q = """
  SELECT ?t1 ?t2
  WHERE {
    ?x1 <http://www.xbrl.org/2003/arcrole#parent-child> [
        xl:from ?t1 ; 
        xl:to ?t2
        ] .
  }"""
parent_child = {str(row[0]): str(row[1]) for row in g.query(q)}

In [11]:
pd.DataFrame(index = parent_child.keys(), data = parent_child.values()).to_csv("finrep_280-parent-child.csv")

## DPM explicit domains

In [12]:
# A domain is a set of elements/values sharing a specified semantic nature. 
# Domain can be of one of two kinds: explicit and typed. 
# An explicit domain has its elements enumerated in the model while a typed domain values 
# are assigned in the reports based on a specified format (data type).

In [13]:
q = """
  SELECT DISTINCT ?t ?x1 ?x2 ?x4 ?x5
  WHERE {
    ?t rdf:type model:explicitDomainType .
    ?t xbrli:periodType ?x1 .
    ?t model:creationDate ?x2 .
    ?t xbrli:nillable ?x4 .
    ?t xbrli:abstract ?x5 .
  }"""

data = [[row[0], concept_labels[str(row[0])]]+list(row[1:]) for row in g.query(q)]
columns = ['Domain uri', 
           'Domain label', 
           'period Type', 
           'creation Date', 
           'nillable', 
           'abstract']
df_domains = pd.DataFrame(data=data,
                          columns=columns)
df_domains.iloc[0:5]

,Domain uri,Domain label,period Type,creation Date,nillable,abstract
0,http://www.eba.europa.eu/xbrl/crr/dict/exp#BA,Defines the basic conceptual meaning of a data...,instant,2013-12-01,true,true
1,http://www.eba.europa.eu/xbrl/crr/dict/exp#MC,Specifies the nature of the item reported (i.e...,instant,2013-12-01,true,true
2,http://www.eba.europa.eu/xbrl/crr/dict/exp#AP,Approach used for the calculation of capital r...,instant,2013-12-01,true,true
3,http://www.eba.europa.eu/xbrl/crr/dict/exp#BT,Dimensions having only two values (usually den...,instant,2013-12-01,true,true
4,http://www.eba.europa.eu/xbrl/crr/dict/exp#CG,Defines the terms and conditions of the collat...,instant,2013-12-01,true,true


In [14]:
print("Number of domains: " +str(len(df_domains)))

Number of domains: 39


## DPM Dimensions

In [15]:
# In order to be used in description of information requirements a domain member 
# or a typed domain value requires a dimensionthat provides a context of its application. 
# In other words dimensions contextualise domain members when applied to a data point
# i.e. they contribute to the semantics of a member which, without a dimension,
# may be insufficient to represent the full meaning of a property.

In [16]:
q = """
  SELECT DISTINCT ?t ?domain
  WHERE {
    ?x2 <http://xbrl.org/int/dim/arcrole#dimension-domain> 
        [ xl:from ?t ;
          xl:to ?domain ] .
  }"""
data = [[row[0], concept_labels[str(row[0])]]+list(row[1:]) for row in g.query(q)]
columns=['Dimension uri', 
         'Dimension label', 
         'Applicable domain code']
df_dimensions = pd.DataFrame(data=data, 
                             columns=columns)
df_dimensions

,Dimension uri,Dimension label,Applicable domain code
0,http://www.eba.europa.eu/xbrl/crr/dict/dim#BAS,Defines the basic conceptual meaning of a data...,http://www.eba.europa.eu/xbrl/crr/dict/exp#BA
1,http://www.eba.europa.eu/xbrl/crr/dict/dim#REF,Specifies the date or the period to which the ...,http://www.eba.europa.eu/xbrl/crr/dict/exp#RF
2,http://www.eba.europa.eu/xbrl/crr/dict/dim#MCY,Specifies the nature of the item reported (i.e...,http://www.eba.europa.eu/xbrl/crr/dict/exp#MC
3,http://www.eba.europa.eu/xbrl/crr/dict/dim#APL,Accounting portfolios shall mean financial ins...,http://www.eba.europa.eu/xbrl/crr/dict/exp#PL
4,http://www.eba.europa.eu/xbrl/crr/dict/dim#ALO,Specifies the type of allowance for credit los...,http://www.eba.europa.eu/xbrl/crr/dict/exp#IM
5,http://www.eba.europa.eu/xbrl/crr/dict/dim#CPS,Defines the sector of the counterparty of fina...,http://www.eba.europa.eu/xbrl/crr/dict/exp#CT
6,http://www.eba.europa.eu/xbrl/crr/dict/dim#IMS,Status for monitoring credit quality of financ...,http://www.eba.europa.eu/xbrl/crr/dict/exp#IM
7,http://www.eba.europa.eu/xbrl/crr/dict/dim#PFS,Identifies the performing status of financial ...,http://www.eba.europa.eu/xbrl/crr/dict/exp#IM
8,http://www.eba.europa.eu/xbrl/crr/dict/dim#REC,For the accumulated other comprehensive income...,http://www.eba.europa.eu/xbrl/crr/dict/exp#BT
9,http://www.eba.europa.eu/xbrl/crr/dict/dim#MCE,Defines the main category that generates the r...,http://www.eba.europa.eu/xbrl/crr/dict/exp#MC


## All domain members of all domains

In [17]:
# Elements of an explicit domain are called domain members. 
# A domain member (or simply a member) is enumerated element of an explicit domain.
# All members from a domain share a certain common nature defined subjectively
# but applied consistently by the model’s author.

In [18]:
# This may take a while
def members(domain):
    q = """
      SELECT DISTINCT ?t ?label
      WHERE {
        ?l <http://xbrl.org/int/dim/arcrole#domain-member>
            [ xl:from <"""+str(domain)+"""> ;
              xl:to ?t ] .
        ?t rdf:type nonnum:domainItemType .
        ?x <http://www.xbrl.org/2003/arcrole#concept-label>
            [ xl:from ?t ;
              xl:to [rdf:value ?label ] ] .
        }"""
    return g.query(q)

df_members = pd.DataFrame()
for domain in df_domains.iloc[:, 0]:
    data = [[urldefrag(domain)[1]]+[urldefrag(row[0])[1]]+list(row[1:]) for row in members(domain)]
    columns = ['Domain',
               'Member',
               'Member label']
    df_members = df_members.append(pd.DataFrame(data=data,
                                                columns=columns))

In [19]:
print("Number of members: "+str(len(df_members)))

Number of members: 6224


## Template structures

In [20]:
namespaces = {"xl": 'http://www.xbrl.org/2003/XLink#',
              "xlink": "http://www.w3.org/1999/xlink#",
              "formula": "http://xbrl.org/2008/formula#",
              "rdf": 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'}
role_rc = rdflib.URIRef("http://www.eurofiling.info/xbrl/role#rc-code")
role_label = rdflib.URIRef("http://www.xbrl.org/2008/role#label")

NameError: name 'rdflib' is not defined

In [ ]:
q_label = rdflib.plugins.sparql.prepareQuery('''
      SELECT DISTINCT ?label
      WHERE {
        ?l <http://xbrl.org/arcrole/2008#element-label> ?s .
        ?s xl:from ?element .
        ?s xl:to [xlink:role ?role ; rdf:value ?label] .
      }''', 
      initNs = namespaces)
q_children = rdflib.plugins.sparql.prepareQuery('''
      SELECT DISTINCT ?t
      WHERE {
        ?s xl:from ?parent .
        ?s xl:to ?t .
      }''',
      initNs = namespaces)
q_concept = rdflib.plugins.sparql.prepareQuery('''
      SELECT DISTINCT ?c
      WHERE {
        OPTIONAL { ?item formula:concept ?c } .
      }''',
      initNs = namespaces)
q_dim = rdflib.plugins.sparql.prepareQuery('''
      SELECT DISTINCT ?d_name ?m_name
      WHERE {
        OPTIONAL { ?item formula:explicitDimension [
                         formula:dimension ?d_name ; 
                         formula:member ?m_name ] }
      }''',
      initNs = namespaces)
q_axis_points = rdflib.plugins.sparql.prepareQuery('''
      SELECT DISTINCT ?t ?axis
      WHERE {
        ?s xl:from ?table_url .
        ?s xl:axis ?axis .
        ?s xl:to ?t .
      }''', 
      initNs = namespaces)

def search_tree(c, depth, axis, data):
    
    if c != []:
        
        for r in g.query(q_children, initBindings={'parent': c}):
            
            element = r[0]

            element = '/'.join(element.split("/")[:-1]) + "#"+element.split("/")[-1].split("#")[-1]
            element = element.replace("eu/fr/", "")
            element = element.replace("f_", "F_")
            element = rdflib.URIRef(element)
            
            rccode = list(g.query(q_label, initBindings={'element': element, 'role': role_rc}))
            if rccode != []:
                rccode = str(rccode[0][0])
            else:
                rccode = None
            label = list(g.query(q_label, initBindings={'element': element, 'role': role_label}))
            if label != []:
                label = str(label[0][0])
            else:
                label = None

            concept_data = list(g.query(q_concept, initBindings={'item': r[0]}))
            if concept_data == []:
                concept_data = [None]

            dim_data = list(g.query(q_dim, initBindings={'item': r[0]}))
            if dim_data == []:
                dim_data = ["", ""]
            else:
                dim_data = [[str(d[0]) for d in dim_data], [str(d[1]) for d in dim_data]]
                
            data.append([str(axis), depth, str(rccode), str(label), concept_data[0]]+dim_data)
            search_tree(r[0], depth+1, axis, data)
    return data

In [ ]:
columns = ['table', 'axis', 'depth', 'rc-code', 'label', 'concept', 'dimension', 'member']
df_tables = pd.DataFrame(columns=columns)

for table in tables:
    table_url = rdflib.URIRef(table)

    axis_points = list()
    for axis in ['x', 'y', 'z']:
        axis_url = rdflib.term.Literal(axis, datatype=rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral'))
        axis_points.append(g.query(q_axis_points, initBindings={'table_url': table_url, 'axis': axis_url}))

    data_x = list()
    for row in axis_points[0]:
        data_x = search_tree(row[0], 0, row[1], data_x)
    data_x = [[table]+line for line in data_x]
    
    data_y = list()
    for row in axis_points[1]:
        data_y = search_tree(row[0], 0, row[1], data_y)
    data_y = [[table]+line for line in data_y]

    data_z = list()
    for row in axis_points[2]:
        data_z = search_tree(row[0], 0, row[1], data_z)
    data_z = [[table]+line for line in data_z]

    df_table = pd.DataFrame(data=data_x+data_y, columns=columns)
    df_table = df_table.sort_values(['axis', 'rc-code']).reset_index(drop=True)
    df_tables = df_tables.append(df_table, ignore_index=True)

In [ ]:
df_tables.to_csv("finrep_280_tables.csv")